In [4]:
!pip install requests

In [77]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import json

In [78]:
## Save list_movie_data in json
def save_data(filename, data):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

## Load file data 
def load_data(filename):
    with open(filename, 'w', encoding='utf-8') as f:
        return json.load(f)
    
#function for scraping infobox
def scrape(url):
    page = requests.get(url)
    toScrape = bs(page.content, 'html.parser')
    movie_details = toScrape.find(class_='infobox vevent')
    rows = movie_details.find_all('tr') 
    movie_data = {}

    movie_data['Title'] = rows[0].find('th').get_text()
    clean_tags(movie_details)
    for i, row in enumerate(rows):
        try:
            if i <= 1:
                continue
            elif row.find('th').get_text() == 'Based on':
                continue
            else:
                column = row.find('th').get_text(' ', strip=True)
                movie_data[column] = clean(row)
        except:
            pass
    return movie_data

#remove troublesome tags
def clean_tags(content):
    t = ['sup', 'span']
    tags = content.find_all(t)
    for tag in tags:
        tag.decompose()

#function to clean data scraped from wikipedia infobox
def clean(row):
    if row.find('th').get_text() == 'Release date':
        date = row.find('td').get_text(' ', strip=True).split('(')[0].replace('\xa0', ' ')
        return date
    elif row.find('br'):
        return [text for text in row.find('td').stripped_strings]
    elif row.find('li'):
        return [li.get_text(' ', strip=True).replace('\xa0', ' ') for li in row.find_all('li')]
    else:
        if '$' in row.find('td').get_text():
            if 'million' in row.find('td').get_text():
                if '-' in row.find('td').get_text():
                    number = float(row.find('td').get_text().split('-')[0].replace('$',''))
                number = float(row.find('td').get_text().split(' ')[0].replace('$',''))
                money = number * (10**6)
                return money
            elif 'billion' in row.find('td').get_text():
                if '-' in row.find('td').get_text():
                    number = float(row.find('td').get_text().split('-')[0].replace('$',''))
                number = float(row.find('td').get_text().split(' ')[0].replace('$',''))
                money = number * (10**9)
                return money
       
    return row.find('td').get_text().strip().replace('\n', ',').replace('\xa0', ' ')

# load in table of WBMovies 
page = requests.get('https://en.wikipedia.org/wiki/List_of_Warner_Bros._films_(2000%E2%80%932009)')
content = bs(page.content, 'html.parser')
table_rows = content.select('.wikitable.sortable i')

#loop through table rows and scrape entry given the url
#Append movie info each iteration
list_movie_data = []
for i,row in enumerate(table_rows):
    if i % 10 == 0:
        print(i);
    try:
        path = row.find('a')['href']
        movie_url = 'https://en.wikipedia.org' + path
        list_movie_data.append(scrape(movie_url))
    except Exception as e:
        print(movie_url)
        print(e)
    
save_data('WB_movie_data.json', list_movie_data)

## still need to change key to 'Running time (min)' then remove 'minutes' from value 
## convert date to datetime object


0
10
20
30
40
https://en.wikipedia.org/wiki/Harry_Potter_and_the_Philosopher%27s_Stone_(film)
'NoneType' object is not subscriptable
50
60
70
80
90
100
110
120
130
140
https://en.wikipedia.org/wiki/The_Dukes_of_Hazzard_(film)
'NoneType' object is not subscriptable
150
160
170
180
190
200
210
220
https://en.wikipedia.orghttps://nl.wikipedia.org/wiki/Morrison_krijgt_een_zusje
HTTPSConnectionPool(host='en.wikipedia.orghttps', port=443): Max retries exceeded with url: //nl.wikipedia.org/wiki/Morrison_krijgt_een_zusje (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002635F4DC640>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
https://en.wikipedia.orghttps://nl.wikipedia.org/wiki/Hoe_overleef_ik_mezelf%3F_(film)
HTTPSConnectionPool(host='en.wikipedia.orghttps', port=443): Max retries exceeded with url: //nl.wikipedia.org/wiki/Hoe_overleef_ik_mezelf%3F_(film) (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object